In [2]:
# from gridfs import GridFS
import pymongo
import redis



MONGODB_URL = f"mongodb://0.0.0.0:27018/"
print("MONGODB_URL: ", MONGODB_URL)




client = pymongo.MongoClient(MONGODB_URL)
db = client["depictioDB"]


data_collection = db["data_collections"]
workflow_collection = db["workflows"]
runs_collection = db["runs"]
files_collection = db["files"]
users_collection = db["users"]
# groups_collection = db["groups_collection"]

MONGODB_URL:  mongodb://0.0.0.0:27018/


In [15]:
# Query in files_collection
files_collection.find_one({"file_location": "/Users/tweber/Data/mosaicatcher-pipeline/2019-12-16-HWVTJAFXY/HG002x02/plots/JBROWSE/HG002x02PE20480-SV.bed.gz"})
# Show all documents from workflow_collection
# for doc in workflow_collection.find():
#     print(doc)

{'_id': ObjectId('65fb5db47a58df9605e71ce2'),
 'file_location': '/Users/tweber/Data/mosaicatcher-pipeline/2019-12-16-HWVTJAFXY/HG002x02/plots/JBROWSE/HG002x02PE20480-SV.bed.gz',
 'S3_location': None,
 'filename': 'HG002x02PE20480-SV.bed.gz',
 'creation_time': datetime.datetime(2024, 1, 29, 10, 7, 6),
 'modification_time': datetime.datetime(2024, 1, 25, 7, 9, 40),
 'data_collection': {'_id': ObjectId('65fb5d9c7a58df9605e6a620'),
  'data_collection_tag': 'SV_calls',
  'description': 'BED files containing SV calls generated by MosaiCatcher',
  'config': {'type': 'JBrowse2',
   'regex': {'pattern': '{cell}-SV.bed.gz',
    'type': 'file-based',
    'wildcards': [{'name': 'cell', 'wildcard_regex': '.*'}]},
   'dc_specific_properties': {'index_extension': 'tbi',
    'jbrowse_template_location': '/Users/tweber/Gits/depictio/CLI_client/configs/mosaicatcher_pipeline/jbrowse2_templates/SV_calls.json'},
   'join': {'on_columns': ['cell'],
    'how': 'inner',
    'with_dc': ['mosaicatcher_stats']}}

In [22]:
# The ID of the data collection you're interested in
from bson import ObjectId


data_collection_id = ObjectId("65fb5f3cdb34e814a1f9084f")

# Query to retrieve all files belonging to the specified data collection
files = files_collection.find({"data_collection._id": data_collection_id})
for file in files[:10]:
    print(file)

{'_id': ObjectId('65fb5f3fdb34e814a1f92606'), 'file_location': '/Users/tweber/Data/mosaicatcher-pipeline/2019-01-02-HGNWMAFXY/SSC11163x02/plots/JBROWSE/SSC11163x02PE20302-SV.bed.gz', 'S3_location': None, 'filename': 'SSC11163x02PE20302-SV.bed.gz', 'creation_time': datetime.datetime(2024, 1, 29, 10, 4, 55), 'modification_time': datetime.datetime(2024, 1, 21, 21, 41, 46), 'data_collection': {'_id': ObjectId('65fb5f3cdb34e814a1f9084f'), 'data_collection_tag': 'SV_calls', 'description': 'BED files containing SV calls generated by MosaiCatcher', 'config': {'type': 'JBrowse2', 'regex': {'pattern': '{cell}-SV.bed.gz', 'type': 'file-based', 'wildcards': [{'name': 'cell', 'wildcard_regex': '.*'}]}, 'dc_specific_properties': {'index_extension': 'tbi', 'jbrowse_template_location': '/Users/tweber/Gits/depictio/CLI_client/configs/mosaicatcher_pipeline/jbrowse2_templates/SV_calls.json'}, 'join': {'on_columns': ['cell'], 'how': 'inner', 'with_dc': ['mosaicatcher_stats']}}}, 'run_id': '2019-01-02-HGNW

In [27]:
# Constructing the nested dictionary
import collections
nested_dict = collections.defaultdict(lambda: collections.defaultdict(dict))

files = files_collection.find({"data_collection._id": data_collection_id})
for file in files:
    for wildcard in file["wildcards"]:
        nested_dict[file["data_collection"]["_id"]][wildcard["name"]][wildcard["value"]] = file["file_location"]

# Example output
print(nested_dict)

defaultdict(<function <lambda> at 0x13382b920>, {ObjectId('65fb5f3cdb34e814a1f9084f'): defaultdict(<class 'dict'>, {'cell': {'SSC11163x02PE20302': '/Users/tweber/Data/mosaicatcher-pipeline/2019-01-02-HGNWMAFXY/SSC11163x02/plots/JBROWSE/SSC11163x02PE20302-SV.bed.gz.tbi', 'SSC11163x02PE20303': '/Users/tweber/Data/mosaicatcher-pipeline/2019-01-02-HGNWMAFXY/SSC11163x02/plots/JBROWSE/SSC11163x02PE20303-SV.bed.gz.tbi', 'SSC11163x02PE20304': '/Users/tweber/Data/mosaicatcher-pipeline/2019-01-02-HGNWMAFXY/SSC11163x02/plots/JBROWSE/SSC11163x02PE20304-SV.bed.gz.tbi', 'SSC11163x02PE20305': '/Users/tweber/Data/mosaicatcher-pipeline/2019-01-02-HGNWMAFXY/SSC11163x02/plots/JBROWSE/SSC11163x02PE20305-SV.bed.gz.tbi', 'SSC11163x02PE20306': '/Users/tweber/Data/mosaicatcher-pipeline/2019-01-02-HGNWMAFXY/SSC11163x02/plots/JBROWSE/SSC11163x02PE20306-SV.bed.gz.tbi', 'SSC11163x02PE20307': '/Users/tweber/Data/mosaicatcher-pipeline/2019-01-02-HGNWMAFXY/SSC11163x02/plots/JBROWSE/SSC11163x02PE20307-SV.bed.gz.tbi',

In [17]:
# id=ObjectId('65fb5bb927b90358513efd08') file_location=PosixPath('/Users/tweber/Data/mosaicatcher-pipeline/2019-09-11-HTN7CAFXY/HG01114x02/plots/JBROWSE/HG01114x02PE20356-SV.bed.gz.tbi') S3_location=None filename='HG01114x02PE20356-SV.bed.gz.tbi' creation_time=datetime.datetime(2024, 1, 29, 10, 5, 44) modification_time=datetime.datetime(2024, 1, 24, 6, 1, 31) data_collection=DataCollection(id=ObjectId('65fb5bb927b90358513efd06'), data_collection_tag='SV_calls', description='BED files containing SV calls generated by MosaiCatcher', config=DataCollectionConfig(type='JBrowse2', regex=Regex(pattern='{cell}-SV.bed.gz', type='file-based', wildcards=[WildcardRegexBase(name='cell', wildcard_regex='.*')]), dc_specific_properties=DCJBrowse2Config(index_extension='tbi', jbrowse_template_location='/Users/tweber/Gits/depictio/CLI_client/configs/mosaicatcher_pipeline/jbrowse2_templates/SV_calls.json'), join=TableJoinConfig(on_columns=['cell'], how='inner', with_dc=['mosaicatcher_stats']))) run_id='2019-09-11-HTN7CAFXY' registration_time=datetime.datetime(2024, 3, 20, 21, 57, 7, 395615) wildcards=[WildcardRegex(name='cell', wildcard_regex='.*', value='HG01114x02PE20356')]
# Find all files with a specific data_collection_id
for file in files_collection.find({"data_collection": {"_id": "65fb5bb927b90358513efd06"}}):
    print(file)